In [296]:
import pandas as pd
import numpy as np
import os

In [297]:
DefHC = pd.read_csv("DefHC.csv", index_col="Index")
WS    = pd.read_csv("WebScrape.csv", index_col="Index")
DNB   = pd.read_csv("DNB.csv", index_col="Index")

In [298]:
print("DefHC: ",DefHC.shape)
print("WS:    ",WS.shape)
print("DNB:   ",DNB.shape)

DefHC:  (4349, 26)
WS:     (7605, 24)
DNB:    (10195, 22)


In [299]:
DefHC['HealthSystem_L1_Name'].value_counts()

Ascension Health               1231
Tenet Healthcare               1158
Sutter Health                   448
Advocate Aurora Health          373
Baylor Scott & White Health     317
Banner Health                   277
NYU Langone Health              275
CommonSpirit Health             147
Genesis Health System           104
Tucson Medical Center            19
Name: HealthSystem_L1_Name, dtype: int64

In [300]:
WS['HealthSystemName'].value_counts()

Ascension Healthcare      2355
Advocate Aurora Health    1708
Baylor Scott & White       854
Sutter Health              699
Quest Diagnostics          603
Tenet Healthcare           593
New York University        521
Banner Health               94
TriHealth                   91
Genesis Health System       70
TMC Healthcare              17
Name: HealthSystemName, dtype: int64

In [301]:
DNB['HealthSystem_L1_Name'].value_counts()

Quest Diagnostics Incorporated                                                                4900
Tenet Healthcare Corporation                                                                  1904
Ascension Health Alliance                                                                      994
Advocate Aurora Health Inc.                                                                    903
Banner Health                                                                                  360
Sutter Health                                                                                  327
Baylor Scott & White Health                                                                    322
New York University                                                                            313
Genesis Health System                                                                           77
Catholic Health Initiatives                                                                     62
TMC Health

In [302]:
DefHC.columns

Index(['HealthSystem_L1_DefHC_ID', 'HealthSystem_L1_Name',
       'HealthSystem_L2_DefHC_ID', 'HealthSystem_L2_Name',
       'HealthSystem_L3_DefHC_ID', 'HealthSystem_L3_Name', 'DefHC_ID',
       'DefHC_Name', 'Other_Name', 'ProvType', 'ProvSubType', 'TaxID', 'NPI',
       'Website', 'Phone', 'AddressScore', 'Address', 'Lat', 'Long', 'Adr1',
       'Adr2', 'City', 'State', 'Zip5', 'Zip4', 'Country'],
      dtype='object')

In [303]:
WS.columns

Index(['HealthSystemID', 'HealthSystemName', 'Name', 'ProviderType',
       'ProviderSubType', 'Phone', 'Fax', 'Orig_Adr1', 'Orig_Adr2',
       'Orig_City', 'Orig_State', 'Orig_Zip5', 'Orig_Zip4', 'AddressScore',
       'Address', 'Lat', 'Long', 'Adr1', 'Adr2', 'City', 'State', 'Zip5',
       'Zip4', 'Country'],
      dtype='object')

In [304]:
DNB.columns

Index(['HealthSystem_L1_DUNS_ID', 'HealthSystem_L1_Name',
       'HealthSystem_L2_DUNS_ID', 'HealthSystem_L2_Name', 'DUNS_ID',
       'DUNS_Name', 'Other_Name', 'Other_Name2', 'ProvType', 'Phone', 'Fax',
       'AddressScore', 'Address', 'Lat', 'Long', 'Adr1', 'Adr2', 'City',
       'State', 'Zip5', 'Zip4', 'Country'],
      dtype='object')

In [305]:
import bamboolib

### Clean: Remove Features,  Dedupe, and ReIndex

#### DefHC

In [306]:
DefHC = DefHC.reset_index()
DefHC_clean = DefHC.drop(columns=['HealthSystem_L1_DefHC_ID', 'HealthSystem_L2_DefHC_ID', 'HealthSystem_L3_DefHC_ID',
                                  'DefHC_ID', 'ProvType', 'ProvSubType', 'TaxID', 'NPI', 'Website', 'AddressScore', 
                                  'Country', 'Address', 'Zip4', 'Index', 'Phone'])

In [307]:
DefHC_clean.shape

(4349, 12)

In [308]:
DefHC_dedupe = DefHC_clean.drop_duplicates(keep='first')
DefHC_dedupe.shape

(1867, 12)

In [309]:
DefHC_dedupe['HealthSystem_L1_Name'].value_counts()

Ascension Health               527
Tenet Healthcare               483
Advocate Aurora Health         185
Baylor Scott & White Health    167
Sutter Health                  163
NYU Langone Health             142
Banner Health                  133
CommonSpirit Health             35
Genesis Health System           28
Tucson Medical Center            4
Name: HealthSystem_L1_Name, dtype: int64

In [310]:
DefHC_dedupe = DefHC_dedupe.reset_index()

In [311]:
DefHC_dedupe = DefHC_dedupe.rename(columns={'level_0': 'DefHC_Index'})
DefHC_dedupe = DefHC_dedupe.drop(columns=['index'])

In [312]:
DefHC_new = DefHC_dedupe

#### DUNS 

In [313]:
DNB = DNB.reset_index()
DNB_clean = DNB.drop(columns=['HealthSystem_L1_DUNS_ID', 'HealthSystem_L2_DUNS_ID', 'Index', 'DUNS_ID', 
                              'ProvType', 'Phone', 'Fax', 'AddressScore', 'Address', 'Zip4', 'Country'])

In [314]:
DNB_clean.shape

(10195, 12)

In [315]:
DNB_dedupe = DNB_clean.drop_duplicates(keep='first')
DNB_dedupe.shape

(9761, 12)

In [316]:
DNB_dedupe['HealthSystem_L1_Name'].value_counts()

Quest Diagnostics Incorporated                                                                4759
Tenet Healthcare Corporation                                                                  1832
Ascension Health Alliance                                                                      946
Advocate Aurora Health Inc.                                                                    863
Banner Health                                                                                  340
Sutter Health                                                                                  302
New York University                                                                            286
Baylor Scott & White Health                                                                    276
Genesis Health System                                                                           71
Catholic Health Initiatives                                                                     60
TMC Health

In [317]:
from matplotlib import pyplot as pl
%matplotlib inline

In [318]:
DNB_dedupe = DNB_dedupe.reset_index()

In [319]:
DNB_dedupe = DNB_dedupe.rename(columns={'index': 'DMB_Index'})

In [320]:
DNB_new = DNB_dedupe

#### WebScrape

In [321]:
WS = WS.reset_index()
WS_clean = WS.drop(columns=['Index', 'HealthSystemID', 'ProviderType', 'ProviderSubType', 'Phone', 'Fax', 
                            'AddressScore', 'Orig_Adr1', 'Orig_Adr2', 'Orig_City', 'Orig_State', 
                            'Orig_Zip5', 'Orig_Zip4', 'Zip4', 'Country'])

In [322]:
WS_dedupe = WS_clean.drop_duplicates(keep='first')
WS_dedupe.shape

(7412, 10)

In [323]:
WS_dedupe['HealthSystemName'].value_counts()

Ascension Healthcare      2284
Advocate Aurora Health    1672
Baylor Scott & White       824
Sutter Health              699
Quest Diagnostics          592
Tenet Healthcare           585
New York University        521
TriHealth                   91
Banner Health               91
Genesis Health System       36
TMC Healthcare              17
Name: HealthSystemName, dtype: int64

In [324]:
WS_dedupe.columns

Index(['HealthSystemName', 'Name', 'Address', 'Lat', 'Long', 'Adr1', 'Adr2',
       'City', 'State', 'Zip5'],
      dtype='object')

In [325]:
WS_clean = WS_dedupe

In [326]:
WS_clean = WS_clean.reset_index()
WS_clean = WS_clean.rename(columns={'index': 'WS_Index'})

### Matching (RecordLinkage) 

In [328]:
import recordlinkage as rl
from   recordlinkage.compare import Geographic
from   recordlinkage.preprocessing import clean, phonetic # Since we will be using Lat/Long and HealthSystem Name, no need to clean data

In [329]:
indexer = rl.Index()
indexer.full()

<Index>

In [330]:
candidates = indexer.index(DNB_clean, DefHC_clean)
print(len(candidates))

44338055


In [331]:
indexer_1 = rl.Index()
indexer_1.block(left_on='State', right_on='State')
candidates_1= indexer_1.index(DNB_clean, DefHC_clean)
print(len(candidates_1))

3080380


In [332]:
indexer_2 = rl.Index()
indexer_2.sortedneighbourhood(left_on='State', right_on='State')
candidates_2 = indexer_2.index(DNB_clean, DefHC_clean)
print(len(candidates_2))

5101701


In [286]:
# Skip
compare = rl.Compare()

compare.geo(left_on_lat = "Lat", left_on_lng = "Long", 
            right_on_lat = "Lat", right_on_lng = "Long", 
            method = "linear", offset = 0.0, 
            scale = 1.0, origin = 0.0, 
            missing_value = 0.0, label = None)

features   = compare.compute(candidates, DNB_clean,DefHC_clean)
features_1 = compare.compute(candidates_1, DNB_clean,DefHC_clean)
features_2 = compare.compute(candidates_2, DNB_clean,DefHC_clean)

In [288]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

1.000000       11913
0.999906          48
0.999887           5
0.999855           1
0.999855           3
              ...   
0.000212           1
0.000185           1
0.000147           1
0.000114           6
0.000000    44259131
Length: 12568, dtype: int64

In [289]:
features_1.sum(axis=1).value_counts().sort_index(ascending=False)

1.000000      11913
0.999906         48
0.999887          5
0.999855          1
0.999855          3
             ...   
0.000212          1
0.000185          1
0.000147          1
0.000114          6
0.000000    3001702
Length: 12438, dtype: int64

In [290]:
features_2.sum(axis=1).value_counts().sort_index(ascending=False)

1.000000      11913
0.999906         48
0.999887          5
0.999855          1
0.999855          3
             ...   
0.000212          1
0.000185          1
0.000147          1
0.000114          6
0.000000    5023023
Length: 12438, dtype: int64

In [333]:
compare = rl.Compare()
compare.exact  ('Lat',  'Lat',  label='Latitude')
compare.exact  ('Long', 'Long', label='Longitude')
compare.string ('HealthSystem_L1_Name',
                'HealthSystem_L1_Name',
                 threshold=0.85,
                 label='HealthSystem_L1_Name')

features   = compare.compute(candidates,   DNB_clean,DefHC_clean)
features_1 = compare.compute(candidates_1, DNB_clean,DefHC_clean)
features_2 = compare.compute(candidates_2, DNB_clean,DefHC_clean)

In [334]:
features = features.reset_index()
features

Latitude  Longitude  HealthSystem_L1_Name
0     0            0          0                   0.0
      1            0          0                   0.0
      2            0          0                   0.0
      3            0          0                   0.0
      4            0          0                   0.0
...              ...        ...                   ...
10194 4344         0          0                   0.0
      4345         0          0                   0.0
      4346         0          0                   0.0
      4347         0          0                   0.0
      4348         0          0                   0.0

[44338055 rows x 3 columns]

In [335]:
features_1 = features_1.reset_index()
features_1

Latitude  Longitude  HealthSystem_L1_Name
0    1446         0          0                   0.0
     1621         0          0                   0.0
     1667         0          0                   0.0
     1668         0          0                   0.0
     1669         0          0                   0.0
...             ...        ...                   ...
6929 293          0          0                   0.0
     294          0          0                   0.0
     295          0          0                   0.0
     1708         0          0                   0.0
     1709         0          0                   0.0

[3080380 rows x 3 columns]

In [336]:
features_2 = features_2.reset_index()
features_2

Latitude  Longitude  HealthSystem_L1_Name
0    3197         0          0                   0.0
     3198         0          0                   0.0
2    3197         0          0                   0.0
     3198         0          0                   0.0
48   3197         0          0                   0.0
...             ...        ...                   ...
7472 1613         0          0                   0.0
     1614         0          0                   0.0
     1615         0          0                   0.0
     1616         0          0                   0.0
     1663         0          0                   0.0

[5101701 rows x 3 columns]

In [337]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

2.0       11913
1.0         280
0.0    44325862
dtype: int64

In [338]:
features_1.sum(axis=1).value_counts().sort_index(ascending=False)

2.0      11913
1.0        280
0.0    3068187
dtype: int64

In [339]:
features_2.sum(axis=1).value_counts().sort_index(ascending=False)


2.0      11913
1.0        280
0.0    5089508
dtype: int64

In [340]:
potential_matches = features[features.sum(axis=1) > 1].reset_index()
potential_matches

,level_0,level_1,Latitude,Longitude,HealthSystem_L1_Name
0,6,4072,1,1,0.0
1,6,4073,1,1,0.0
2,6,4074,1,1,0.0
3,6,4075,1,1,0.0
4,6,4076,1,1,0.0
...,...,...,...,...,...
11908,10191,2215,1,1,0.0
11909,10191,2297,1,1,0.0
11910,10192,2267,1,1,0.0
11911,10194,1448,1,1,0.0


In [365]:
DNB_new.loc[7353,:]

DMB_Index                                                            7631
HealthSystem_L1_Name    Ascension Health Alliance                     ...
HealthSystem_L2_Name    Ascension Health Alliance                     ...
DUNS_Name               Clifton Park Internal Medicine                ...
Other_Name                                                            ...
Other_Name2                                                           ...
Lat                                                              -73.7888
Long                                                              42.8651
Adr1                                                     1 Tallow Wood Dr
Adr2                                                                  NaN
City                                                         Clifton Park
State                                                                  NY
Zip5                                                                12065
Name: 7353, dtype: object

In [364]:
DefHC_clean.loc[418,:]

HealthSystem_L1_Name                     Baylor Scott & White Health
HealthSystem_L2_Name                                             NaN
HealthSystem_L3_Name                                             NaN
DefHC_Name              Baylor Scott & White Medical Center - Temple
Other_Name                                                       NaN
Lat                                                         -97.3658
Long                                                         31.0789
Adr1                                                  2401 S 31st St
Adr2                                                             NaN
City                                                          Temple
State                                                             TX
Zip5                                                           76504
Name: 418, dtype: object